<a href="https://colab.research.google.com/github/piyushjain220/TSAI/blob/main/NLP/Session4/Sentiment_Analysis_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 - Updated Sentiment Analysis: Code Updated for Assignment 4

**Change this code in such a way that:**

- it has 3 LSTM layers
- it has used a for loop to do so in the forward function
- the dropout value used is 0.2
- trained on the text that is reversed (for example "my name is Rohan" becomes "Rohan is name my"
- achieves 87% or more accuracy
- once done, share the Github link as well (after training on Google Colab, move the file to GitHub).


In [1]:
import torch
from torchtext import data
from torchtext import datasets
from functools import partial
from tqdm import tqdm 

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)
tqdm = partial(tqdm, position=0, leave=True)

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [3]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [4]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [5]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

## Build the Model

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super(RNN,self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn1 = nn.LSTM(embedding_dim, int(hidden_dim/4), bidirectional=bidirectional)        
        self.rnn2 = nn.LSTM(int(hidden_dim/2), int(hidden_dim/4), bidirectional=bidirectional)        
        self.rnn3 = nn.LSTM(int(hidden_dim/2), hidden_dim, bidirectional=bidirectional)
        
        self.rnns = [self.rnn1,self.rnn2,self.rnn3]
        self.fc = nn.Linear(hidden_dim*2 , output_dim)        
        self.dropout = nn.Dropout(dropout)       
        
    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu().numpy())
        
        pack_emb = packed_embedded
        for lstm in self.rnns:
          packed_output, (hidden, cell) = lstm(pack_emb)
          pack_emb = packed_output

        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        return self.fc(hidden)

In [7]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,475,561 trainable parameters


In [9]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1830, -1.1160,  2.3601,  ..., -1.6465,  0.2431,  0.5666],
        [-0.2326, -1.2659, -1.1949,  ...,  0.9952, -0.0324, -0.1769],
        [-0.6184,  0.0806,  1.7381,  ...,  0.0445, -0.5604, -0.3227]])


## Train the Model

In [10]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)

In [11]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [12]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [13]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for _, batch in tqdm(enumerate(iterator), total=len(iterator)):
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text

        reverse_text = torch.flip(text, [0])

        combined_text = torch.cat([text, reverse_text], axis=1)
        combined_labels = torch.cat([batch.label, batch.label])
        combined_lengths = torch.cat([text_lengths, text_lengths])
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [14]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for _, batch in tqdm(enumerate(iterator), total=len(iterator)):

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

And also create a nice function to tell us how long our epochs are taking.

In [15]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [16]:
print(model)

RNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (rnn1): LSTM(100, 64, bidirectional=True)
  (rnn2): LSTM(128, 64, bidirectional=True)
  (rnn3): LSTM(128, 256, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


Finally, we train our model...

In [17]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  0%|          | 0/274 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 26s
	Train Loss: 0.635 | Train Acc: 62.25%
	 Val. Loss: 0.622 |  Val. Acc: 66.17%


  1%|          | 2/274 [00:00<00:15, 17.52it/s]

Epoch: 02 | Epoch Time: 0m 27s
	Train Loss: 0.509 | Train Acc: 75.32%
	 Val. Loss: 0.391 |  Val. Acc: 83.55%


  1%|          | 2/274 [00:00<00:22, 11.85it/s]

...

Epoch: 18 | Epoch Time: 0m 27s
	Train Loss: 0.061 | Train Acc: 98.03%
	 Val. Loss: 0.377 |  Val. Acc: 88.70%


  0%|          | 1/274 [00:00<00:30,  8.92it/s]

Epoch: 19 | Epoch Time: 0m 27s
	Train Loss: 0.051 | Train Acc: 98.51%
	 Val. Loss: 0.417 |  Val. Acc: 87.93%


100%|██████████| 118/118 [00:04<00:00, 27.18it/s]

Epoch: 20 | Epoch Time: 0m 26s
	Train Loss: 0.054 | Train Acc: 98.20%
	 Val. Loss: 0.445 |  Val. Acc: 87.85%


In [18]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

100%|██████████| 391/391 [00:13<00:00, 28.65it/s]

Test Loss: 0.295 | Test Acc: 88.20%


## User Input

In [19]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

An example negative review...

In [20]:
predict_sentiment(model, "This film is terrible")

0.016976863145828247

An example positive review...

In [21]:
predict_sentiment(model, "This film is great")

0.9975478053092957

## Next Steps

We've now built a decent sentiment analysis model for movie reviews! In the next notebook we'll implement a model that gets comparable accuracy with far fewer parameters and trains much, much faster.